In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path

class InteractiveNHLExplorer:
    """Interactive explorer using ipywidgets with inline backend"""
    
    def __init__(self, processed_data_dir="data/processed"):
        self.processed_data_dir = Path(processed_data_dir)
        self.data = self._load_all_data()
        self.current_game_data = None
        
    def _load_all_data(self):
        """Load all processed data"""
        all_dfs = []
        csv_files = list(self.processed_data_dir.rglob("all_*_games.csv"))
        
        for csv_file in csv_files:
            try:
                df = pd.read_csv(csv_file)
                parts = csv_file.parts
                df['season_folder'] = parts[-3]
                df['game_type'] = parts[-2]
                all_dfs.append(df)
            except Exception as e:
                print(f"Error loading {csv_file}: {e}")
        
        return pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame()
    
    def create_interactive_widget(self):
        """Create the main interactive widget"""
        if self.data.empty:
            print("No data loaded. Please check your processed data directory.")
            return
        
        print(f"Loaded {len(self.data)} events")
        
        # Create widgets
        seasons = sorted(self.data['season_folder'].unique())
        game_types = ['general', 'playoff']
        
        self.season_dropdown = widgets.Dropdown(
            options=seasons,
            value=seasons[0],
            description='Season:',
            style={'description_width': 'initial'}
        )
        
        self.game_type_dropdown = widgets.Dropdown(
            options=game_types,
            value='general',
            description='Game Type:',
            style={'description_width': 'initial'}
        )
        
        self.game_dropdown = widgets.Dropdown(
            description='Game ID:',
            style={'description_width': 'initial'}
        )
        
        self.event_slider = widgets.IntSlider(
            value=0,
            min=0,
            max=100,
            step=1,
            description='Event Index:',
            continuous_update=False,
            style={'description_width': 'initial'}
        )
        
        self.output = widgets.Output()
        
        # Set up event handlers
        def on_season_change(change):
            self._update_game_list()
            self._plot_current_event()
            
        def on_game_type_change(change):
            self._update_game_list()
            self._plot_current_event()
            
        def on_game_change(change):
            self._update_event_slider()
            self._plot_current_event()
            
        def on_event_change(change):
            self._plot_current_event()
        
        # Link events to handlers
        self.season_dropdown.observe(on_season_change, names='value')
        self.game_type_dropdown.observe(on_game_type_change, names='value')
        self.game_dropdown.observe(on_game_change, names='value')
        self.event_slider.observe(on_event_change, names='value')
        
        # Display widgets
        display(widgets.VBox([
            widgets.HBox([self.season_dropdown, self.game_type_dropdown]),
            self.game_dropdown,
            self.event_slider,
            self.output
        ]))
        
        # Initial setup
        self._update_game_list()
    
    def _update_game_list(self):
        """Update game list based on current selections"""
        season = self.season_dropdown.value
        game_type = self.game_type_dropdown.value
        
        games = self.data[
            (self.data['season_folder'] == season) & 
            (self.data['game_type'] == game_type)
        ]['game_id'].unique()
        
        self.game_dropdown.options = sorted(games)
        if len(games) > 0:
            self.game_dropdown.value = games[0]
        else:
            self.game_dropdown.options = []
            self.game_dropdown.value = None
    
    def _update_event_slider(self):
        """Update event slider range based on selected game"""
        game_id = self.game_dropdown.value
        if game_id:
            game_data = self.data[self.data['game_id'] == game_id]
            self.current_game_data = game_data.reset_index(drop=True)
            max_events = len(game_data) - 1 if len(game_data) > 0 else 0
            self.event_slider.max = max_events
            self.event_slider.value = 0
        else:
            self.current_game_data = None
            self.event_slider.max = 0
            self.event_slider.value = 0
    
    def _plot_current_event(self):
        """Plot the current selected event"""
        with self.output:
            clear_output(wait=True)
            
            if (self.current_game_data is None or 
                len(self.current_game_data) == 0 or 
                self.game_dropdown.value is None):
                print("No game data available. Please select a game.")
                return
            
            event_index = self.event_slider.value
            if event_index >= len(self.current_game_data):
                event_index = len(self.current_game_data) - 1
                self.event_slider.value = event_index
            
            event = self.current_game_data.iloc[event_index]
            
            # Create the interactive plot
            fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))
            
            # Plot 1: Single event on rink
            self._draw_ice_rink(ax1)
            self._plot_single_event(ax1, event, event_index)
            
            # Plot 2: All events in game
            self._plot_all_events(ax2)
            
            # Plot 3: Event information
            self._display_event_info(ax3, event, event_index)
            
            plt.tight_layout()
            plt.show()
    
    def _draw_ice_rink(self, ax):
        """Draw NHL ice rink"""
        ax.set_xlim(-100, 100)
        ax.set_ylim(-42.5, 42.5)
        ax.set_aspect('equal')
        
        # Rink boundary
        rink = patches.Rectangle((-100, -42.5), 200, 85, linewidth=2, 
                               edgecolor='black', facecolor='lightblue', alpha=0.3)
        ax.add_patch(rink)
        
        # Lines
        ax.axvline(x=-25, color='blue', linestyle='-', alpha=0.5)
        ax.axvline(x=25, color='blue', linestyle='-', alpha=0.5)
        ax.axvline(x=-89, color='red', linestyle='-', alpha=0.7)
        ax.axvline(x=89, color='red', linestyle='-', alpha=0.7)
        
        # Nets
        left_net = patches.Rectangle((-89.5, -3), 1, 6, linewidth=2, 
                                   edgecolor='red', facecolor='red', alpha=0.5)
        right_net = patches.Rectangle((88.5, -3), 1, 6, linewidth=2, 
                                    edgecolor='red', facecolor='red', alpha=0.5)
        ax.add_patch(left_net)
        ax.add_patch(right_net)
        
        ax.set_title('Current Event Location', fontsize=12)
        ax.set_xlabel('Distance from Center (feet)')
        ax.set_ylabel('Distance from Center (feet)')
        ax.grid(True, alpha=0.3)
    
    def _plot_single_event(self, ax, event, event_index):
        """Plot a single event on the rink"""
        x, y = event['x_coord'], event['y_coord']
        
        if pd.notna(x) and pd.notna(y):
            color = 'red' if event['event_type'] == 'GOAL' else 'blue'
            marker = '*' if event['event_type'] == 'GOAL' else 'o'
            size = 150 if event['event_type'] == 'GOAL' else 100
            
            ax.scatter(x, y, c=color, marker=marker, s=size, alpha=0.8, 
                      edgecolors='black', linewidth=1.5)
            
            # Add annotation
            ax.annotate(f"Event {event_index + 1}\n{event['shot_type']}", 
                       xy=(x, y), xytext=(x + 5, y + 5),
                       fontsize=10, bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.9),
                       arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=0.1"))
        else:
            ax.text(0, 0, "No coordinates\navailable", ha='center', va='center',
                   fontsize=12, bbox=dict(boxstyle="round,pad=0.5", facecolor="yellow", alpha=0.8))
    
    def _plot_all_events(self, ax):
        """Plot all events in the current game"""
        if self.current_game_data is None:
            return
            
        self._draw_ice_rink(ax)
        
        shots = self.current_game_data[self.current_game_data['event_type'] == 'SHOT_ON_GOAL']
        goals = self.current_game_data[self.current_game_data['event_type'] == 'GOAL']
        
        if not shots.empty:
            ax.scatter(shots['x_coord'], shots['y_coord'], 
                      c='blue', alpha=0.4, s=30, label=f'Shots ({len(shots)})')
        
        if not goals.empty:
            ax.scatter(goals['x_coord'], goals['y_coord'], 
                      c='red', alpha=0.8, s=60, marker='*', label=f'Goals ({len(goals)})')
        
        ax.legend()
        ax.set_title('All Events in Game', fontsize=12)
    
    def _display_event_info(self, ax, event, event_index):
        """Display detailed event information"""
        ax.axis('off')
        
        if self.current_game_data is not None:
            total_events = len(self.current_game_data)
            current_event = event_index + 1
            
            info_text = f"""
EVENT {current_event} of {total_events}

GAME INFO:
Season: {event['season_folder']}
Game Type: {event['game_type']}
Game ID: {event['game_id']}

EVENT DETAILS:
Type: {event['event_type']}
Period: {event['period']}
Time: {event['period_time']}
Team: {event['team_name']}
Player: {event['player_name']}
Goalie: {event['goalie_name']}
Shot Type: {event.get('shot_type', 'N/A')}

COORDINATES:
X: {event.get('x_coord', 'N/A')}
Y: {event.get('y_coord', 'N/A')}
Distance: {event.get('distance_from_net', 'N/A'):.1f} ft

GAME SUMMARY:
Shots: {len(self.current_game_data[self.current_game_data['event_type'] == 'SHOT_ON_GOAL'])}
Goals: {len(self.current_game_data[self.current_game_data['event_type'] == 'GOAL'])}
            """
        else:
            info_text = "No game data available"
        
        ax.text(0.05, 0.95, info_text, transform=ax.transAxes, fontsize=10, 
               verticalalignment='top', fontfamily='monospace',
               bbox=dict(boxstyle="round,pad=1", facecolor="lightgray", alpha=0.8))
        
        ax.set_title('Event Information', fontsize=12, pad=20)

def launch_interactive_tool():
    """Launch the interactive tool"""
    explorer = InteractiveNHLExplorer("data/processed")
    explorer.create_interactive_widget()
    return explorer

In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
from interactive_tool import launch_interactive_tool

plt.rcParams['figure.figsize'] = [18, 5]
plt.rcParams['font.size'] = 10
print("NHL Interactive Explorer - Ready!")

NHL Interactive Explorer - Ready!


In [13]:
# This should launch the interactive widget
explorer = launch_interactive_tool()